# 💳 SmartSentry AML — Notebook 2: Transaction Generator
---
**Prerequisites:** Run Notebook 1 first, or load the CSVs from `./outputs/`.

**Purpose:** Generate the transactions table with:
- Legitimate transactions (normal behaviour)
- 5 fraud typologies injected
- Full PK/FK linkage to customers, accounts, devices, beneficiaries
- 30+ derived features including rolling velocity, time windows, and risk signals

**PK / FK schema for transactions:**

| Column | Type | Links to |
|---|---|---|
| `transaction_id` | PK | — |
| `customer_id` | FK | `customers.customer_id` |
| `sender_account_id` | FK | `accounts.account_id` |
| `receiver_account_id` | FK (nullable) | `accounts.account_id` — internal transfers only |
| `beneficiary_id` | FK (nullable) | `beneficiaries.beneficiary_id` — external transfers only |
| `device_id` | FK | `devices.device_id` |

> **NULL / XOR rule:** Exactly one of `receiver_account_id` / `beneficiary_id` is non-NULL per row.
> Internal transfer → `receiver_account_id` set. External transfer → `beneficiary_id` set.

In [1]:
# ── CELL 1: Imports ──────────────────────────────────────────────────────
import random
import warnings
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

print("✅ Libraries loaded")

✅ Libraries loaded


In [2]:
# ── CELL 2: Configuration ────────────────────────────────────────────────
# All parameters for transaction generation live here.

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# ─── Simulation Window ────────────────────────────────────────────────────
START_DATE = datetime(2025, 9, 1)
END_DATE   = datetime(2025, 12, 31)
DATE_RANGE_DAYS = (END_DATE - START_DATE).days   # 181 days

# ─── Transaction Volume ───────────────────────────────────────────────────
TARGET_FRAUD_RATE    = 0.04   # 4% overall fraud prevalence
AVG_TXNS_PER_ACCOUNT = 50     # Poisson lambda

# ─── Channels & Transaction Types ─────────────────────────────────────────
CHANNELS      = ["mobile", "web", "branch", "atm"]
CHANNEL_PROBS = [0.50,     0.30,  0.10,    0.10]

# Transaction types available per channel
TXN_TYPES_BY_CHANNEL = {
    "mobile" : ["UPI", "IMPS", "wallet_transfer"],
    "web"    : ["NEFT", "RTGS", "online_transfer"],
    "branch" : ["cash_deposit", "cash_withdrawal", "DD"],
    "atm"    : ["cash_withdrawal", "balance_enquiry"],
}

# These transaction types involve physical cash → cash_flag = 1
CASH_TXN_TYPES = {"cash_deposit", "cash_withdrawal", "DD"}

# ─── Amount Parameters by KYC Level ──────────────────────────────────────
# KYC level is a proxy for customer wealth
# Lognormal: median ≈ exp(mean)
#   low    → median ≈ ₹1,097
#   medium → median ≈ ₹2,981
#   high   → median ≈ ₹13,360
AMOUNT_BY_KYC = {
    "low"    : {"mean": 7.0, "sigma": 1.0},
    "medium" : {"mean": 8.0, "sigma": 1.0},
    "high"   : {"mean": 9.5, "sigma": 1.0},
}
AMOUNT_DEFAULT = {"mean": 8.0, "sigma": 1.0}

# 40% of transactions are internal (account to account)
# 60% are external (account to beneficiary)
INTERNAL_TRANSFER_PROB = 0.40

# ─── Transaction Timing ───────────────────────────────────────────────────
# Hour-of-day weights (index 0=midnight … 23=11pm)
# Normal: peaks around business hours
# Fraud : peaks late night / early morning (less monitoring)
HOUR_WEIGHTS_NORMAL = [1,1,1,1,1,1,2,3,5,6,6,6,5,6,6,6,5,5,4,4,3,2,2,1]
HOUR_WEIGHTS_FRAUD  = [4,4,4,4,3,2,1,1,1,1,1,1,1,1,1,1,2,2,3,3,4,4,4,4]

# ─── Fraud Typology Volumes ────────────────────────────────────────────────
FRAUD_VOLUMES = {
    "mule_ring": {
        "num_rings"        : 60,    # Number of distinct mule rings
        "accounts_per_ring": 10,    # Mule accounts per ring
        "txns_per_account" : 8,     # Transactions per mule
        "day_min"          : 60,    "day_max": 120,
        "amount_mean"      : 9.5,   "amount_sigma": 0.5,
        "channels"         : ["mobile", "web"],
        # All mules share ONE device → key detection signal
    },
    "layering": {
        "num_chains"       : 900,   # Number of layering chains
        "chain_length"     : 3,     # Hops per chain (A→B→C→D)
        "day_min"          : 100,   "day_max": 150,
        "hop_min_mins"     : 5,     "hop_max_mins": 30,
        "amount_mean_base" : 9.0,   "amount_mean_decay": 0.2,
        "amount_sigma"     : 0.8,
        "channels"         : ["mobile", "web"],
        # Amount decreases per hop (laundering fees)
        # All hops are INTERNAL → receiver_account_id set, beneficiary_id NULL
    },
    "ATO": {
        "count"            : 2000,
        "day_min"          : 120,   "day_max": 180,
        "amount_mean"      : 10.5,  "amount_sigma": 0.8,
        "channels"         : ["mobile", "web"],
        # Key signal: device_id ≠ account's home device
    },
    "smurfing": {
        "num_groups"       : 400,
        "min_txns"         : 4,     "max_txns": 8,
        "amount_min"       : 8500,  "amount_max": 9999,   # Just below ₹10k threshold
        "day_window"       : 2,     # Spread over N days
        "channels"         : ["branch", "atm"],
        "force_cash_flag"  : True,  # Always cash
    },
    "identity_fraud": {
        "count"            : 800,
        "max_account_days" : 60,    # Only targets new accounts
        "day_min"          : 0,     "day_max": 30,
        "amount_mean"      : 11.0,  "amount_sigma": 0.6,
        "channels"         : ["mobile", "web"],
    },
}

print("✅ Transaction configuration loaded")
print(f"   Simulation window : {START_DATE.date()} → {END_DATE.date()} ({DATE_RANGE_DAYS} days)")
print(f"   Target fraud rate : {TARGET_FRAUD_RATE:.0%}")

✅ Transaction configuration loaded
   Simulation window : 2025-09-01 → 2025-12-31 (121 days)
   Target fraud rate : 4%


In [3]:
# ── CELL 3: Load Reference Tables ────────────────────────────────────────
# Load the four tables generated in Notebook 1.
# If you ran Notebook 1 in the same session, these are already in memory —
# this cell reloads them from CSV so the notebook is self-contained.

customers     = pd.read_csv(OUTPUT_DIR / "customers.csv")
accounts      = pd.read_csv(OUTPUT_DIR / "accounts.csv")
devices       = pd.read_csv(OUTPUT_DIR / "devices.csv")
beneficiaries = pd.read_csv(OUTPUT_DIR / "beneficiaries.csv")

# Rebuild lookup dicts
device_list = devices["device_id"].tolist()
account_device_map = {acc: random.choice(device_list) for acc in accounts["account_id"]}

bene_list = beneficiaries["beneficiary_id"].tolist()
account_beneficiaries_map = {
    acc: random.sample(bene_list, k=random.randint(2, 5))
    for acc in accounts["account_id"]
}

high_risk_mask = (
    beneficiaries["beneficiary_type"].isin({"crypto", "offshore"}) |
    (beneficiaries["beneficiary_country_risk"] == "high")
)
high_risk_bene_pool = beneficiaries.loc[high_risk_mask, "beneficiary_id"].tolist()

all_account_ids = accounts["account_id"].tolist()

print("✅ Reference tables loaded")
print(f"   customers: {len(customers):,} | accounts: {len(accounts):,} | "
      f"devices: {len(devices):,} | beneficiaries: {len(beneficiaries):,}")
print(f"   high_risk_bene_pool: {len(high_risk_bene_pool):,}")

✅ Reference tables loaded
   customers: 1,000 | accounts: 1,500 | devices: 800 | beneficiaries: 500
   high_risk_bene_pool: 73


In [4]:
# ── CELL 4: Shared Helper Functions ──────────────────────────────────────

def weighted_hour(fraud=False):
    """Sample hour (0–23) weighted by transaction behaviour profile."""
    weights = HOUR_WEIGHTS_FRAUD if fraud else HOUR_WEIGHTS_NORMAL
    return random.choices(range(24), weights=weights)[0]

def random_ts(fraud=False, day_min=0, day_max=None):
    """Random datetime within simulation window."""
    if day_max is None:
        day_max = DATE_RANGE_DAYS
    return START_DATE + timedelta(
        days=random.randint(day_min, min(day_max, DATE_RANGE_DAYS)),
        hours=weighted_hour(fraud),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59),
    )

def get_txn_type(channel):
    """Random transaction type for a given channel."""
    return random.choice(TXN_TYPES_BY_CHANNEL.get(channel, ["online_transfer"]))

def is_cash(txn_type):
    """1 if transaction involves physical cash, else 0."""
    return int(txn_type in CASH_TXN_TYPES)

# Canonical column order — ALL transaction builders must follow this.
TXN_COLS = [
    "transaction_id",        # PK
    "customer_id",           # FK → customers
    "sender_account_id",     # FK → accounts
    "receiver_account_id",   # FK → accounts  (nullable — internal only)
    "beneficiary_id",        # FK → beneficiaries  (nullable — external only)
    "device_id",             # FK → devices
    "timestamp",
    "amount",
    "channel",
    "debit_credit",
    "transaction_type",
    "cash_flag",
    "fraud_type",
    "label",
]

def build_row(txn_id, customer_id, sender_account_id, device_id,
              ts, amount, channel, fraud_type, label,
              receiver_account_id=None, beneficiary_id=None):
    """
    Build one canonical transaction row.
    XOR constraint: exactly one of receiver_account_id / beneficiary_id is non-None.
    """
    txn_type = get_txn_type(channel)
    return [
        f"T{txn_id}",        # transaction_id   PK
        customer_id,          # customer_id      FK → customers
        sender_account_id,    # sender_account_id FK → accounts
        receiver_account_id,  # receiver_account_id (nullable)
        beneficiary_id,       # beneficiary_id (nullable)
        device_id,            # device_id        FK → devices
        ts,                   # timestamp
        round(amount, 2),     # amount (INR)
        channel,              # channel
        "debit",              # debit_credit
        txn_type,             # transaction_type
        is_cash(txn_type),    # cash_flag
        fraud_type,           # fraud_type
        label,                # label
    ]

print("✅ Helper functions defined")
print(f"   TXN_COLS ({len(TXN_COLS)} columns): {TXN_COLS}")

✅ Helper functions defined
   TXN_COLS (14 columns): ['transaction_id', 'customer_id', 'sender_account_id', 'receiver_account_id', 'beneficiary_id', 'device_id', 'timestamp', 'amount', 'channel', 'debit_credit', 'transaction_type', 'cash_flag', 'fraud_type', 'label']


In [5]:
# ── CELL 5: Generate Legitimate Transactions ─────────────────────────────
# Each account sends ~50 transactions (Poisson distributed).
# 40% are internal (account-to-account), 60% external (to beneficiary).
# Amount is drawn from a lognormal distribution keyed to KYC level.
#
# FK linkage per row:
#   customer_id       ← accounts.customer_id (denormalised — single join)
#   sender_account_id ← the account being processed
#   device_id         ← account_device_map[account_id]
#   receiver_account_id OR beneficiary_id ← based on transfer type

acc_meta = accounts.set_index("account_id")
legit_rows = []
txn_id = 0

for acc in accounts["account_id"]:
    kyc    = acc_meta.loc[acc, "kyc_level"]
    cust   = acc_meta.loc[acc, "customer_id"]
    device = account_device_map[acc]
    params = AMOUNT_BY_KYC.get(kyc, AMOUNT_DEFAULT)
    n_txns = np.random.poisson(AVG_TXNS_PER_ACCOUNT)

    for _ in range(n_txns):
        ts     = random_ts(fraud=False)
        amount = np.random.lognormal(mean=params["mean"], sigma=params["sigma"])
        ch     = random.choices(CHANNELS, weights=CHANNEL_PROBS)[0]

        if random.random() < INTERNAL_TRANSFER_PROB:
            legit_rows.append(build_row(
                txn_id, cust, acc, device, ts, amount, ch, "normal", 0,
                receiver_account_id=random.choice(all_account_ids),
                beneficiary_id=None,
            ))
        else:
            legit_rows.append(build_row(
                txn_id, cust, acc, device, ts, amount, ch, "normal", 0,
                receiver_account_id=None,
                beneficiary_id=random.choice(account_beneficiaries_map[acc]),
            ))
        txn_id += 1

legit_df = pd.DataFrame(legit_rows, columns=TXN_COLS)

print(f"✅ Legitimate transactions: {len(legit_df):,} rows")
print(f"   Internal transfers     : {legit_df['receiver_account_id'].notna().sum():,} "
      f"({legit_df['receiver_account_id'].notna().mean()*100:.1f}%)")
print(f"   External transfers     : {legit_df['beneficiary_id'].notna().sum():,} "
      f"({legit_df['beneficiary_id'].notna().mean()*100:.1f}%)")
print()
legit_df.head(3)

✅ Legitimate transactions: 75,083 rows
   Internal transfers     : 30,188 (40.2%)
   External transfers     : 44,895 (59.8%)



,transaction_id,customer_id,sender_account_id,receiver_account_id,beneficiary_id,device_id,timestamp,amount,channel,debit_credit,transaction_type,cash_flag,fraud_type,label
0,T0,C863,A0,A1263,None,D654,2025-11-30 17:30:14,2095.80,atm,debit,balance_enquiry,0,normal,0
1,T1,C863,A0,A665,None,D654,2025-12-26 11:11:44,5029.27,atm,debit,balance_enquiry,0,normal,0
2,T2,C863,A0,None,B326,D654,2025-11-02 02:30:07,867.70,mobile,debit,IMPS,0,normal,0


In [6]:
# ── CELL 6: Inject Fraud Typologies ──────────────────────────────────────
# Five typologies injected in sequence. Each is self-contained below.
# All fraud rows are appended to fraud_rows list.

acc_meta_fraud = accounts.set_index("account_id")
all_accs       = accounts["account_id"].tolist()
all_devs       = devices["device_id"].tolist()
fraud_rows     = []

# ── 6A: MULE RINGS ────────────────────────────────────────────────────────
# Multiple accounts share ONE device and funnel to one high-risk exit bene.
# Graph pattern: star topology (N accounts → 1 beneficiary)
cfg = FRAUD_VOLUMES["mule_ring"]
for _ in range(cfg["num_rings"]):
    mule_accs   = random.sample(all_accs, cfg["accounts_per_ring"])
    shared_dev  = random.choice(all_devs)          # ← shared device
    exit_bene   = random.choice(high_risk_bene_pool)

    for acc in mule_accs:
        cust = acc_meta_fraud.loc[acc, "customer_id"]
        for _ in range(cfg["txns_per_account"]):
            ts  = random_ts(True, cfg["day_min"], cfg["day_max"])
            amt = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
            ch  = random.choice(cfg["channels"])
            fraud_rows.append(build_row(
                txn_id, cust, acc, shared_dev, ts, amt, ch,
                "mule_ring", 1, receiver_account_id=None, beneficiary_id=exit_bene,
            ))
            txn_id += 1

print(f"  Mule rings    : {len([r for r in fraud_rows if r[12]=='mule_ring']):>6,} rows")

## ── 6B: LAYERING CHAINS ───────────────────────────────────────────────────
# Funds hop through 3–4 accounts in minutes. ALL hops are internal.
# Graph pattern: circular chain A→B→C→A (funds return to origin)
cfg = FRAUD_VOLUMES["layering"]
for _ in range(cfg["num_chains"]):
    chain   = random.sample(all_accs, cfg["chain_length"])  # no +1; last node will be chain[0]
    chain   = chain + [chain[0]]                            # close the loop: A→B→C→A
    base_ts = random_ts(True, cfg["day_min"], cfg["day_max"])
    for i in range(cfg["chain_length"]):
        sender   = chain[i]
        receiver = chain[i + 1]   # final hop: C → A ✅
        cust     = acc_meta_fraud.loc[sender, "customer_id"]
        device   = account_device_map[sender]
        ts       = base_ts + timedelta(minutes=random.randint(cfg["hop_min_mins"], cfg["hop_max_mins"]) * (i+1))
        amt      = np.random.lognormal(cfg["amount_mean_base"] - i*cfg["amount_mean_decay"], cfg["amount_sigma"])
        ch       = random.choice(cfg["channels"])
        fraud_rows.append(build_row(
            txn_id, cust, sender, device, ts, amt, ch,
            "layering", 1, receiver_account_id=receiver, beneficiary_id=None,
        ))
        txn_id += 1
print(f"  Layering      : {len([r for r in fraud_rows if r[12]=='layering']):>6,} rows")

# ── 6C: ACCOUNT TAKEOVER (ATO) ────────────────────────────────────────────
# Key signal: device_id is a NEW device (not account's home device)
cfg = FRAUD_VOLUMES["ATO"]
for _ in range(cfg["count"]):
    acc      = random.choice(all_accs)
    cust     = acc_meta_fraud.loc[acc, "customer_id"]
    new_dev  = random.choice(all_devs)   # ← different from home device
    exit_bene = random.choice(high_risk_bene_pool)
    ts       = random_ts(True, cfg["day_min"], cfg["day_max"])
    amt      = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
    ch       = random.choice(cfg["channels"])
    fraud_rows.append(build_row(
        txn_id, cust, acc, new_dev, ts, amt, ch,
        "ATO", 1, receiver_account_id=None, beneficiary_id=exit_bene,
    ))
    txn_id += 1

print(f"  ATO           : {len([r for r in fraud_rows if r[12]=='ATO']):>6,} rows")

# ── 6D: SMURFING / STRUCTURING ────────────────────────────────────────────
# Cash transactions just below ₹10,000 threshold, spread over 2-3 days.
# cash_flag forced to 1.
cfg = FRAUD_VOLUMES["smurfing"]
for _ in range(cfg["num_groups"]):
    acc      = random.choice(all_accs)
    cust     = acc_meta_fraud.loc[acc, "customer_id"]
    device   = account_device_map[acc]
    exit_bene = random.choice(high_risk_bene_pool)
    base_day = random.randint(0, DATE_RANGE_DAYS - cfg["day_window"] - 1)

    for _ in range(random.randint(cfg["min_txns"], cfg["max_txns"])):
        ts  = random_ts(True, day_min=base_day, day_max=base_day + cfg["day_window"])
        amt = round(random.uniform(cfg["amount_min"], cfg["amount_max"]), 2)
        ch  = random.choice(cfg["channels"])
        row = build_row(
            txn_id, cust, acc, device, ts, amt, ch,
            "smurfing", 1, receiver_account_id=None, beneficiary_id=exit_bene,
        )
        if cfg["force_cash_flag"]:
            row[TXN_COLS.index("cash_flag")] = 1   # force cash_flag
        fraud_rows.append(row)
        txn_id += 1

print(f"  Smurfing      : {len([r for r in fraud_rows if r[12]=='smurfing']):>6,} rows")

# ── 6E: IDENTITY FRAUD ────────────────────────────────────────────────────
# New accounts (<60 days) make immediate large transfers to high-risk benes.
cfg = FRAUD_VOLUMES["identity_fraud"]
new_accs = accounts.loc[accounts["account_open_days"] < cfg["max_account_days"], "account_id"].tolist()
sample_n = min(cfg["count"], len(new_accs))

for acc in random.sample(new_accs, sample_n):
    cust     = acc_meta_fraud.loc[acc, "customer_id"]
    device   = account_device_map[acc]
    exit_bene = random.choice(high_risk_bene_pool)
    ts       = random_ts(True, cfg["day_min"], cfg["day_max"])
    amt      = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
    ch       = random.choice(cfg["channels"])
    fraud_rows.append(build_row(
        txn_id, cust, acc, device, ts, amt, ch,
        "identity_fraud", 1, receiver_account_id=None, beneficiary_id=exit_bene,
    ))
    txn_id += 1

print(f"  Identity fraud: {len([r for r in fraud_rows if r[12]=='identity_fraud']):>6,} rows")

fraud_df = pd.DataFrame(fraud_rows, columns=TXN_COLS)
print(f"\n✅ Total fraud rows (pre-rebalance): {len(fraud_df):,}")

  Mule rings    :  4,800 rows
  Layering      :  2,700 rows
  ATO           :  2,000 rows
  Smurfing      :  2,387 rows
  Identity fraud:     22 rows

✅ Total fraud rows (pre-rebalance): 11,909


In [7]:
# ── CELL 7: Merge & Rebalance to Target Fraud Rate ───────────────────────
# Combines legit and fraud, then oversamples fraud with new PKs if needed.

combined      = pd.concat([legit_df, fraud_df], ignore_index=True)
current_fraud = int(combined["label"].sum())
current_total = len(combined)
current_rate  = current_fraud / current_total

print(f"Before rebalancing: {current_rate:.4f} fraud rate ({current_fraud:,} / {current_total:,})")

if current_rate < TARGET_FRAUD_RATE:
    needed = int(TARGET_FRAUD_RATE * current_total / (1 - TARGET_FRAUD_RATE)) - current_fraud
    extra  = fraud_df.sample(n=needed, replace=True, random_state=42).copy()
    extra["transaction_id"] = [f"T{txn_id + i}_OS" for i in range(len(extra))]
    combined = pd.concat([combined, extra], ignore_index=True)
    print(f"Oversampled {needed:,} fraud rows to reach {TARGET_FRAUD_RATE:.0%} target.")

raw_txns = combined.sample(frac=1, random_state=42).reset_index(drop=True)
raw_txns["timestamp"] = pd.to_datetime(raw_txns["timestamp"])

final_rate = raw_txns["label"].mean()
print(f"\n✅ Final: {final_rate:.4f} fraud rate ({int(raw_txns['label'].sum()):,} / {len(raw_txns):,})")
print()
raw_txns["fraud_type"].value_counts()

Before rebalancing: 0.1369 fraud rate (11,909 / 86,992)

✅ Final: 0.1369 fraud rate (11,909 / 86,992)



fraud_type
normal            75083
mule_ring          4800
layering           2700
smurfing           2387
ATO                2000
identity_fraud       22
Name: count, dtype: int64

In [8]:
# ── CELL 8: Entity Enrichment (FK Joins) ─────────────────────────────────
# Join account, device, and beneficiary attributes onto transactions.
# All joins use the explicit FK columns — no positional assumptions.
#
# After this cell, transactions contain all entity-level features
# needed for rule evaluation and ML feature engineering.

txns = raw_txns.copy()

# Join account features (via sender_account_id FK)
# This single join brings in customer attributes too (denormalised in accounts)
acc_join_cols = [
    "account_id", "avg_balance", "account_open_days",
    "kyc_level", "country_risk", "income_bracket",
    "customer_risk_rating", "pep_flag", "occupation",
    "industry", "account_type",
]
txns = txns.merge(
    accounts[acc_join_cols].rename(columns={"account_id": "sender_account_id"}),
    on="sender_account_id", how="left"
)
print(f"After account join   : {txns.shape[1]} columns")

# Join device features (via device_id FK)
dev_cols = ["device_id", "device_age_days", "rooted_flag", "os_type", "vpn_flag", "emulator_flag"]
txns = txns.merge(devices[dev_cols], on="device_id", how="left")
print(f"After device join    : {txns.shape[1]} columns")

# Join beneficiary features (via beneficiary_id FK — nullable)
# Internal transfers will have NaN for beneficiary columns — expected.
bene_cols = ["beneficiary_id", "beneficiary_type", "beneficiary_country_risk"]
txns = txns.merge(beneficiaries[bene_cols], on="beneficiary_id", how="left")
print(f"After bene join      : {txns.shape[1]} columns")

print(f"\n✅ Entity enrichment complete: {len(txns):,} rows × {txns.shape[1]} cols")
print()
txns[["transaction_id","customer_id","sender_account_id","device_id",
      "beneficiary_id","kyc_level","customer_risk_rating","pep_flag",
      "beneficiary_type","amount","label"]].head(3)

After account join   : 24 columns
After device join    : 29 columns
After bene join      : 31 columns

✅ Entity enrichment complete: 86,992 rows × 31 cols



,transaction_id,customer_id,sender_account_id,device_id,beneficiary_id,kyc_level,customer_risk_rating,pep_flag,beneficiary_type,amount,label
0,T12331,C934,A246,D462,None,high,medium,0,NaN,66637.25,0
1,T3955,C247,A79,D284,B203,medium,very_high,0,individual,3758.75,0
2,T71370,C539,A1426,D763,None,high,high,0,NaN,12527.15,0


In [9]:
txns.head()

,transaction_id,customer_id,sender_account_id,receiver_account_id,beneficiary_id,device_id,timestamp,amount,channel,debit_credit,...,occupation,industry,account_type,device_age_days,rooted_flag,os_type,vpn_flag,emulator_flag,beneficiary_type,beneficiary_country_risk
0,T12331,C934,A246,A463,None,D462,2025-10-14 18:22:11,66637.25,web,debit,...,student,manufacturing,business,1169,0,android,0,0,NaN,NaN
1,T3955,C247,A79,None,B203,D284,2025-10-03 11:41:05,3758.75,atm,debit,...,salaried,technology,savings,57,0,ios,0,0,individual,low
2,T71370,C539,A1426,A221,None,D763,2025-10-14 15:04:14,12527.15,mobile,debit,...,self_employed,finance,savings,900,0,ios,0,0,NaN,NaN
3,T48019,C708,A959,A682,None,D484,2025-09-21 06:40:19,1869.35,mobile,debit,...,business_owner,real_estate,retail,662,0,android,0,0,NaN,NaN
4,T36352,C998,A724,None,B295,D154,2025-09-01 09:12:55,678.07,mobile,debit,...,government_employee,retail,retail,1418,0,unknown,0,0,individual,low


In [10]:
# ── CELL 9: Temporal Features ────────────────────────────────────────────
# Derived from timestamp. These are foundational for all velocity features.

txns["timestamp"]    = pd.to_datetime(txns["timestamp"])
txns["hour"]         = txns["timestamp"].dt.hour
txns["day_of_week"]  = txns["timestamp"].dt.dayofweek   # 0=Mon, 6=Sun
txns["date"]         = txns["timestamp"].dt.date
txns["month"]        = txns["timestamp"].dt.month
txns["week_of_year"] = txns["timestamp"].dt.isocalendar().week.astype(int)
txns["is_night"]     = ((txns["hour"] < 6) | (txns["hour"] >= 22)).astype(int)
txns["is_weekend"]   = (txns["day_of_week"] >= 5).astype(int)
txns["is_business_hours"] = ((txns["hour"] >= 9) & (txns["hour"] <= 17)).astype(int)

print(f"✅ Temporal features added: {txns.shape[1]} cols")
print(f"   Night transactions  : {txns['is_night'].sum():,} ({txns['is_night'].mean()*100:.1f}%)")
print(f"   Weekend transactions: {txns['is_weekend'].sum():,} ({txns['is_weekend'].mean()*100:.1f}%)")
print()
txns[["transaction_id","timestamp","hour","day_of_week","is_night","is_weekend","is_business_hours"]].head(4)

✅ Temporal features added: 39 cols
   Night transactions  : 14,188 (16.3%)
   Weekend transactions: 23,759 (27.3%)



,transaction_id,timestamp,hour,day_of_week,is_night,is_weekend,is_business_hours
0,T12331,2025-10-14 18:22:11,18,1,0,0,0
1,T3955,2025-10-03 11:41:05,11,4,0,0,1
2,T71370,2025-10-14 15:04:14,15,1,0,0,1
3,T48019,2025-09-21 06:40:19,6,6,0,1,0


In [11]:
# ── CELL 10: Amount & Balance Ratio Features ─────────────────────────────

# Amount to balance ratio — how large is this txn relative to typical balance?
# Values > 1 mean the transaction exceeds the account's average balance.
txns["amount_to_balance_ratio"] = (
    txns["amount"] / txns["avg_balance"].clip(lower=1)
).round(4)

# Is the amount a 'round number'? Round amounts can indicate structuring.
txns["is_round_amount"] = ((txns["amount"] % 1000 == 0) & (txns["amount"] >= 1000)).astype(int)

# Log-transformed amount (useful for ML models)
txns["log_amount"] = np.log1p(txns["amount"]).round(4)

# High-risk beneficiary flag
hr_pool_set = set(high_risk_bene_pool)
txns["high_risk_beneficiary"] = txns["beneficiary_id"].isin(hr_pool_set).astype(int)

# Account age at time of transaction (more precise than static account_open_days)
txns["account_age_days"] = (
    txns["timestamp"] - pd.to_datetime(START_DATE - timedelta(days=1)) +
    pd.to_timedelta(txns["account_open_days"], unit="D")
).dt.days.clip(lower=1)

print(f"✅ Amount & ratio features added: {txns.shape[1]} cols")
print()
txns[["transaction_id","amount","avg_balance","amount_to_balance_ratio",
      "is_round_amount","log_amount","account_age_days"]].describe().round(3)

✅ Amount & ratio features added: 44 cols



,amount,avg_balance,amount_to_balance_ratio,is_round_amount,log_amount,account_age_days
count,86992.000,86992.000,86992.000,86992.0,86992.000,86992.000
mean,10464.330,35009.423,0.830,0.0,8.398,1099.207
std,19388.688,43182.553,2.513,0.0,1.340,577.058
min,28.930,382.200,0.000,0.0,3.399,31.000
25%,1714.082,10942.110,0.065,0.0,7.447,600.000
50%,4558.465,21371.330,0.208,0.0,8.425,1104.000
75%,11095.935,41485.320,0.657,0.0,9.314,1608.000
max,777489.750,477149.690,104.604,0.0,13.564,2121.000


In [13]:
# ── CELL 11: Account-Level Velocity Features (Rolling Windows) ───────────
# These features answer: "how active has this ACCOUNT been recently?"
#
# Features computed per sender_account_id:
#   txn_count_last_1hr    — transactions in the last 1 hour
#   txn_count_last_24hr   — transactions in the last 24 hours
#   total_amount_last_24hr — total INR sent in the last 24 hours
#   txn_count_last_7d     — transactions in the last 7 days
#   txn_count_last_30d    — transactions in the last 30 days
#   total_amount_last_7d  — total amount in last 7 days
#   total_amount_last_30d — total amount in last 30 days
#   amount_zscore_30d     — z-score vs 30-day rolling mean (anomaly signal)
#   dormancy_flag         — 1 if > 30 days since last transaction

print("Computing account-level rolling velocity features...")

txns_sorted = txns.sort_values(["sender_account_id", "timestamp"]).copy()
txns_sorted = txns_sorted.set_index("timestamp")

def account_rolling(group):
    group = group.sort_index()
    amt   = group["amount"]

    group["txn_count_last_1hr"]     = amt.rolling("1H").count().astype(int)
    group["txn_count_last_24hr"]    = amt.rolling("24H").count().astype(int)
    group["total_amount_last_24hr"] = amt.rolling("24H").sum().round(2)
    group["total_amount_last_7d"]   = amt.rolling("7D").sum().round(2)
    group["total_amount_last_30d"]  = amt.rolling("30D").sum().round(2)

    # # Z-score: how anomalous is this amount vs recent history?
    # mu  = amt.rolling("30D").mean()
    # sig = amt.rolling("30D").std().fillna(1).clip(lower=1e-6)
    # group["amount_zscore_30d"] = ((amt - mu) / sig).round(4)

    # Dormancy: >30 days since last transaction from this account
    prev_ts = group.index.to_series().shift(1)
    gap_days = (group.index.to_series() - prev_ts).dt.total_seconds() / 86400
    group["dormancy_flag"] = (gap_days > 30).fillna(False).astype(int)

    # Cumulative count (overall activity level)
    group["txn_velocity_cumulative"] = range(1, len(group) + 1)

    return group

txns_sorted = (
    txns_sorted
    .groupby("sender_account_id", group_keys=False)
    .apply(account_rolling)
    .reset_index()
)

print(f"✅ Account velocity features added: {txns_sorted.shape[1]} cols")
print()
vel_cols = ["transaction_id","sender_account_id","amount",
            "txn_count_last_1hr","txn_count_last_24hr","total_amount_last_24hr","dormancy_flag"]
txns_sorted[vel_cols].head(6)

Computing account-level rolling velocity features...
✅ Account velocity features added: 51 cols



,transaction_id,sender_account_id,amount,txn_count_last_1hr,txn_count_last_24hr,total_amount_last_24hr,dormancy_flag
0,T27,A0,819.18,1,1,819.18,0
1,T44,A0,691.84,1,1,691.84,0
2,T46,A0,1546.30,1,1,1546.30,0
3,T31,A0,380.81,1,2,1927.11,0
4,T19,A0,875.00,1,1,875.00,0
5,T18,A0,4748.81,1,2,5623.81,0


In [25]:
# ── CELL 12: Customer-Level Velocity Features ────────────────────────────
# These features answer: "how active has this CUSTOMER been recently?"
# A customer may hold multiple accounts — this aggregates across all of them.
#
# Features computed per customer_id:
#   cust_txn_count_last_1hr     — transactions across all accounts in 1 hour
#   cust_txn_count_last_24hr    — transactions across all accounts in 24 hours
#   cust_total_amount_last_24hr — total INR across all accounts in 24 hours
#   cust_txn_count_last_7d      — transactions in last 7 days
#   cust_total_amount_last_7d   — total amount in last 7 days

print("Computing customer-level rolling velocity features...")

cust_sorted = txns_sorted.sort_values(["customer_id", "timestamp"]).copy()
cust_sorted = cust_sorted.set_index("timestamp")

def customer_rolling(group):
    group = group.sort_index()
    amt   = group["amount"]

    group["cust_txn_count_last_1hr"]     = amt.rolling("1H").count().astype(int)
    group["cust_txn_count_last_24hr"]    = amt.rolling("24H").count().astype(int)
    group["cust_total_amount_last_24hr"] = amt.rolling("24H").sum().round(2)
    group["cust_total_amount_last_30d"]  = amt.rolling("30D").sum().round(2)

    return group  # ✅ string column handled separately below

cust_sorted = (
    cust_sorted
    .groupby("customer_id", group_keys=False)
    .apply(customer_rolling)
    .reset_index()
)

# ── Unique accounts used by customer in last 30 days ─────────────────────
# Computed separately because rolling/expanding cannot handle string columns
cust_sorted = cust_sorted.sort_values(["customer_id", "timestamp"])

def unique_accounts_30d(group):
    group = group.set_index("timestamp").sort_index()
    result = []
    times  = group.index
    accs   = group["sender_account_id"].values
    for i, t in enumerate(times):
        cutoff = t - pd.Timedelta("30D")
        window = accs[(times >= cutoff) & (times <= t)]
        result.append(len(set(window)))
    group["cust_unique_accounts_30d"] = result
    return group.reset_index()

cust_sorted = (
    cust_sorted
    .groupby("customer_id", group_keys=False)
    .apply(unique_accounts_30d)
)

print(f"✅ Customer velocity features added: {cust_sorted.shape[1]} cols")
print()
cust_vel_cols = ["transaction_id","customer_id","sender_account_id","amount",
                 "cust_txn_count_last_1hr","cust_txn_count_last_24hr",
                 "cust_total_amount_last_24hr","cust_unique_accounts_30d"]
cust_sorted[cust_vel_cols].head(5)

Computing customer-level rolling velocity features...
✅ Customer velocity features added: 56 cols



,transaction_id,customer_id,sender_account_id,amount,cust_txn_count_last_1hr,cust_txn_count_last_24hr,cust_total_amount_last_24hr,cust_unique_accounts_30d
0,T67865,C0,A1353,917.51,1,1,917.51,1
1,T55261,C0,A1100,1152.79,1,2,2070.30,2
2,T67895,C0,A1353,332.39,1,1,332.39,2
3,T67877,C0,A1353,906.33,2,2,1238.72,2
4,T55267,C0,A1100,147.78,1,1,147.78,2


In [26]:
# ── CELL 13: AML Rule Engine (50 Rules) ──────────────────────────────────
# Each rule produces a binary column (rule_<name>) = 0 or 1.
# Three aggregate columns summarise rule activity per transaction:
#   rule_trigger_count    — total rules fired
#   max_rule_severity     — severity (1–3) of most severe rule fired
#   weighted_rule_score   — sum of severities of all fired rules
#
# Severity: 1=low  2=medium  3=high

# Format: (name, severity, lambda row → bool)
RULES = [
    # Amount-based
    ("large_cash_deposit",         3, lambda r: r["cash_flag"]==1 and r["amount"]>50_000),
    ("cash_just_below_threshold",  3, lambda r: r["cash_flag"]==1 and 8_000<=r["amount"]<=9_999),
    ("high_value_transfer",        2, lambda r: r["amount"]>100_000),
    ("micro_transaction",          1, lambda r: r["amount"]<10),
    ("round_amount_large",         1, lambda r: r.get("is_round_amount",0)==1 and r["amount"]>=10_000),

    # Time-based
    ("night_transaction",          2, lambda r: r.get("is_night",0)==1),
    ("weekend_high_value",         2, lambda r: r.get("is_weekend",0)==1 and r["amount"]>20_000),
    ("dormant_account_activation", 3, lambda r: r.get("dormancy_flag",0)==1 and r["amount"]>5_000),

    # Velocity (account-level)
    ("high_acct_velocity_1hr",     3, lambda r: r.get("txn_count_last_1hr",0)>5),
    ("high_acct_velocity_24hr",    2, lambda r: r.get("txn_count_last_24hr",0)>20),
    ("high_acct_velocity_7d",      2, lambda r: r.get("txn_count_last_7d",0)>60),
    ("high_acct_volume_24hr",      2, lambda r: r.get("total_amount_last_24hr",0)>500_000),
    ("high_acct_volume_7d",        2, lambda r: r.get("total_amount_last_7d",0)>2_000_000),
    ("amount_spike_30d",           3, lambda r: r.get("amount_zscore_30d",0)>4),
    ("rapid_burst",                3, lambda r: r.get("txn_count_last_1hr",0)>3),

    # Velocity (customer-level)
    ("high_cust_velocity_1hr",     3, lambda r: r.get("cust_txn_count_last_1hr",0)>8),
    ("high_cust_velocity_24hr",    2, lambda r: r.get("cust_txn_count_last_24hr",0)>30),
    ("high_cust_volume_24hr",      2, lambda r: r.get("cust_total_amount_last_24hr",0)>1_000_000),
    ("high_cust_volume_7d",        2, lambda r: r.get("cust_total_amount_last_7d",0)>3_000_000),

    # Account / KYC
    ("low_kyc_high_amount",        3, lambda r: r.get("kyc_level")=="low" and r["amount"]>30_000),
    ("new_account_large_txn",      3, lambda r: r.get("account_open_days",9999)<60 and r["amount"]>10_000),
    ("high_amount_to_balance",     2, lambda r: r.get("amount_to_balance_ratio",0)>3),
    ("very_high_risk_customer",    3, lambda r: r.get("customer_risk_rating")=="very_high"),
    ("pep_high_value",             3, lambda r: r.get("pep_flag",0)==1 and r["amount"]>10_000),
    ("high_risk_country_sender",   2, lambda r: r.get("country_risk")=="high"),
    ("corporate_large_cash",       2, lambda r: r.get("account_type") in ("corporate","business")
                                                 and r["cash_flag"]==1 and r["amount"]>20_000),

    # Beneficiary
    ("high_risk_beneficiary",      3, lambda r: r.get("high_risk_beneficiary",0)==1),
    ("crypto_transfer",            2, lambda r: r.get("beneficiary_type")=="crypto"),
    ("offshore_transfer",          3, lambda r: r.get("beneficiary_type")=="offshore"),
    ("high_risk_bene_country",     2, lambda r: r.get("beneficiary_country_risk")=="high"),

    # Device / channel
    ("rooted_device",              2, lambda r: r.get("rooted_flag",0)==1),
    ("vpn_proxy_detected",         2, lambda r: r.get("vpn_flag",0)==1),
    ("emulator_detected",          3, lambda r: r.get("emulator_flag",0)==1),
    ("new_device_large_txn",       1, lambda r: r.get("device_age_days",9999)<90 and r["amount"]>10_000),
    ("atm_high_withdrawal",        2, lambda r: r["channel"]=="atm" and r["amount"]>20_000),
    ("branch_night_txn",           2, lambda r: r["channel"]=="branch" and r.get("is_night",0)==1),

    # Structuring
    ("structuring_pattern",        3, lambda r: r["cash_flag"]==1 and 8_500<=r["amount"]<=9_999),
    ("multiple_small_cash",        3, lambda r: r["cash_flag"]==1
                                                 and r.get("txn_count_last_7d",0)>5 and r["amount"]<10_000),

    # Industry / occupation
    ("high_risk_industry",         1, lambda r: r.get("industry") in ("real_estate","construction","unknown")),
    ("student_high_value",         2, lambda r: r.get("occupation")=="student" and r["amount"]>50_000),
    ("unemployed_large_transfer",  2, lambda r: r.get("occupation")=="unemployed" and r["amount"]>20_000),
    ("freelancer_offshore",        2, lambda r: r.get("occupation")=="freelancer"
                                                 and r.get("beneficiary_type")=="offshore"),

    # Combined risk combos
    ("pep_crypto_transfer",        3, lambda r: r.get("pep_flag",0)==1 and r.get("beneficiary_type")=="crypto"),
    ("very_high_risk_offshore",    3, lambda r: r.get("customer_risk_rating")=="very_high"
                                                 and r.get("beneficiary_type")=="offshore"),
    ("low_kyc_offshore",           3, lambda r: r.get("kyc_level")=="low" and r.get("beneficiary_type")=="offshore"),
    ("low_income_large_txn",       2, lambda r: r.get("income_bracket")=="low" and r["amount"]>50_000),
    ("vpn_offshore",               3, lambda r: r.get("vpn_flag",0)==1 and r.get("beneficiary_type")=="offshore"),
    ("emulator_crypto",            3, lambda r: r.get("emulator_flag",0)==1 and r.get("beneficiary_type")=="crypto"),
    ("new_account_offshore",       3, lambda r: r.get("account_open_days",9999)<90
                                                 and r.get("beneficiary_type")=="offshore"),
    ("new_acct_high_cust_velocity",3, lambda r: r.get("account_open_days",9999)<90
                                                 and r.get("cust_txn_count_last_24hr",0)>10),
]

result = cust_sorted.copy()
severity_map = {}

for rule_name, severity, condition in RULES:
    col = f"rule_{rule_name}"
    try:
        result[col] = result.apply(lambda row, c=condition: int(bool(c(row))), axis=1)
    except Exception:
        result[col] = 0
    severity_map[col] = severity

rule_cols   = [f"rule_{n}" for n, _, _ in RULES]
sev_weights = np.array([severity_map[c] for c in rule_cols])
rule_matrix = result[rule_cols].values

result["rule_trigger_count"]  = rule_matrix.sum(axis=1).astype(int)
result["max_rule_severity"]   = (rule_matrix * sev_weights).max(axis=1).astype(int)
result["weighted_rule_score"] = (rule_matrix * sev_weights).sum(axis=1).astype(int)

total_firings = int(result["rule_trigger_count"].sum())
print(f"✅ Rule engine complete")
print(f"   {len(RULES)} rules | {total_firings:,} total firings | "
      f"avg {result['rule_trigger_count'].mean():.2f} per transaction")
print()
result[["transaction_id","label","fraud_type","rule_trigger_count",
        "max_rule_severity","weighted_rule_score"]].head(5)

✅ Rule engine complete
   50 rules | 119,162 total firings | avg 1.37 per transaction



,transaction_id,label,fraud_type,rule_trigger_count,max_rule_severity,weighted_rule_score
0,T67865,0,normal,1,2,2
1,T55261,0,normal,0,0,0
2,T67895,0,normal,2,3,5
3,T67877,0,normal,0,0,0
4,T55267,0,normal,0,0,0


In [27]:
# ── CELL 14: Final Transactions Table ────────────────────────────────────

final_txns = result.copy()

print("══════════════════════════════════════════════════════════════")
print("  FINAL TRANSACTIONS TABLE SUMMARY")
print("══════════════════════════════════════════════════════════════")
print(f"  Rows             : {len(final_txns):,}")
print(f"  Columns          : {final_txns.shape[1]}")
print(f"  Fraud rate       : {final_txns['label'].mean():.4f}")
print()
print("  PK / FK columns:")
pk_fk_cols = ["transaction_id","customer_id","sender_account_id",
              "receiver_account_id","beneficiary_id","device_id"]
for col in pk_fk_cols:
    null_pct = final_txns[col].isna().mean() * 100
    print(f"    {col:<30} {null_pct:.1f}% null")

print()
print("  XOR check (receiver_account_id XOR beneficiary_id):")
both    = (final_txns["receiver_account_id"].notna() & final_txns["beneficiary_id"].notna()).sum()
neither = (final_txns["receiver_account_id"].isna()  & final_txns["beneficiary_id"].isna()).sum()
print(f"    Rows with BOTH set   : {both}  {'✅' if both==0 else '❌'}")
print(f"    Rows with NEITHER set: {neither}  {'✅' if neither==0 else '❌'}")

print()
print("  Feature groups:")
base_cols  = [c for c in TXN_COLS if c in final_txns.columns]
enrich     = ["avg_balance","account_open_days","kyc_level","pep_flag",
              "device_age_days","rooted_flag","vpn_flag","emulator_flag",
              "beneficiary_type","beneficiary_country_risk"]
temporal   = ["hour","day_of_week","is_night","is_weekend","is_business_hours","month"]
ratio      = ["amount_to_balance_ratio","is_round_amount","log_amount",
              "high_risk_beneficiary","account_age_days"]
acct_vel   = [c for c in final_txns.columns if c.startswith("txn_") or c.startswith("total_amount_last") or c=="amount_zscore_30d" or c=="dormancy_flag"]
cust_vel   = [c for c in final_txns.columns if c.startswith("cust_")]
rule_agg   = ["rule_trigger_count","max_rule_severity","weighted_rule_score"]
rule_flags = [c for c in final_txns.columns if c.startswith("rule_") and c not in rule_agg]

for group_name, cols in [
    ("Core transaction", base_cols),
    ("Entity enrichment", [c for c in enrich if c in final_txns.columns]),
    ("Temporal", temporal),
    ("Amount / ratios", ratio),
    ("Account velocity", acct_vel),
    ("Customer velocity", cust_vel),
    ("Rule aggregates", rule_agg),
    ("Rule flags", rule_flags),
]:
    print(f"    {group_name:<20} : {len(cols)} columns")

══════════════════════════════════════════════════════════════
  FINAL TRANSACTIONS TABLE SUMMARY
══════════════════════════════════════════════════════════════
  Rows             : 86,992
  Columns          : 109
  Fraud rate       : 0.1369

  PK / FK columns:
    transaction_id                 0.0% null
    customer_id                    0.0% null
    sender_account_id              0.0% null
    receiver_account_id            62.2% null
    beneficiary_id                 37.8% null
    device_id                      0.0% null

  XOR check (receiver_account_id XOR beneficiary_id):
    Rows with BOTH set   : 0  ✅
    Rows with NEITHER set: 0  ✅

  Feature groups:
    Core transaction     : 14 columns
    Entity enrichment    : 10 columns
    Temporal             : 6 columns
    Amount / ratios      : 5 columns
    Account velocity     : 7 columns
    Customer velocity    : 5 columns
    Rule aggregates      : 3 columns
    Rule flags           : 50 columns


In [28]:
print("DONE with the Process")

DONE with the Process


In [29]:
# ── CELL 15: Complete Feature Catalogue ──────────────────────────────────

FULL_CATALOGUE = {
    # ── CORE TRANSACTION FIELDS ───────────────────────────────────────────
    "transaction_id":           "PK. Unique transaction identifier. Format: T0, T1, …",
    "customer_id":              "FK → customers.customer_id. Denormalised for single-key joins.",
    "sender_account_id":        "FK → accounts.account_id. Account that initiated the payment.",
    "receiver_account_id":      "FK → accounts.account_id. NULLABLE. Set for internal transfers only.",
    "beneficiary_id":           "FK → beneficiaries.beneficiary_id. NULLABLE. Set for external transfers only.",
    "device_id":                "FK → devices.device_id. Device used to initiate transaction.",
    "timestamp":                "Full datetime of transaction (UTC).",
    "amount":                   "Transaction amount in INR. Lognormally distributed.",
    "channel":                  "Channel: mobile | web | branch | atm.",
    "debit_credit":             "Always 'debit' — sender perspective.",
    "transaction_type":         "Payment instrument: UPI | IMPS | wallet_transfer | NEFT | RTGS | online_transfer | cash_deposit | cash_withdrawal | DD | balance_enquiry.",
    "cash_flag":                "1 = physical cash involved (cash_deposit, cash_withdrawal, DD).",
    "fraud_type":               "Ground truth typology: normal | mule_ring | layering | ATO | smurfing | identity_fraud.",
    "label":                    "Target variable. 0 = legitimate, 1 = fraud.",

    # ── ENTITY ENRICHMENT (from FK joins) ─────────────────────────────────
    "avg_balance":              "From accounts. Average account balance (INR). Denominator for ratio features.",
    "account_open_days":        "From accounts. Days since account opened. <60 days = elevated identity fraud risk.",
    "kyc_level":                "From accounts/customers. KYC depth: low | medium | high.",
    "country_risk":             "From accounts/customers. Home jurisdiction risk: low | medium | high.",
    "income_bracket":           "From accounts/customers. Declared income: low | medium | high.",
    "customer_risk_rating":     "From accounts/customers. CDD rating: low | medium | high | very_high.",
    "pep_flag":                 "From accounts/customers. 1 = Politically Exposed Person.",
    "occupation":               "From accounts/customers. Declared occupation.",
    "industry":                 "From accounts/customers. Business sector.",
    "account_type":             "From accounts/customers. Account type: retail | corporate | savings | current | business.",
    "device_age_days":          "From devices. Age of device in days.",
    "rooted_flag":              "From devices. 1 = rooted/jailbroken device.",
    "os_type":                  "From devices. android | ios | windows | unknown.",
    "vpn_flag":                 "From devices. 1 = VPN/proxy detected.",
    "emulator_flag":            "From devices. 1 = emulated environment (not physical device).",
    "beneficiary_type":         "From beneficiaries. individual | merchant | crypto | offshore.",
    "beneficiary_country_risk": "From beneficiaries. Jurisdiction risk: low | medium | high.",

    # ── TEMPORAL FEATURES ─────────────────────────────────────────────────
    "hour":                     "Hour of day (0–23). Fraud peaks at 0–4 and 20–23.",
    "day_of_week":              "Day of week (0=Mon … 6=Sun).",
    "date":                     "Calendar date of transaction.",
    "month":                    "Calendar month (1–12).",
    "week_of_year":             "ISO week number (1–53).",
    "is_night":                 "1 = transaction between 22:00 and 05:59.",
    "is_weekend":               "1 = Saturday or Sunday.",
    "is_business_hours":        "1 = transaction between 09:00 and 17:00.",

    # ── AMOUNT & RATIO FEATURES ────────────────────────────────────────────
    "amount_to_balance_ratio":  "amount / avg_balance (clipped at 1). >1 means txn exceeds typical balance.",
    "is_round_amount":          "1 = amount is a round multiple of ₹1,000 and ≥ ₹1,000.",
    "log_amount":               "log1p(amount). Log-transformed for ML models.",
    "high_risk_beneficiary":    "1 = beneficiary_id is in the high-risk pool (crypto/offshore/high-country-risk).",
    "account_age_days":         "Account age in days at the time of this transaction (dynamic, not static).",

    # ── ACCOUNT-LEVEL VELOCITY (per sender_account_id) ────────────────────
    "txn_count_last_1hr":       "ACCOUNT: Number of transactions sent in the last 1 hour.",
    "txn_count_last_24hr":      "ACCOUNT: Number of transactions sent in the last 24 hours.",
    "total_amount_last_24hr":   "ACCOUNT: Total INR sent in the last 24 hours.",
    "txn_count_last_7d":        "ACCOUNT: Transaction count in the last 7 days.",
    "txn_count_last_30d":       "ACCOUNT: Transaction count in the last 30 days.",
    "total_amount_last_7d":     "ACCOUNT: Total INR sent in the last 7 days.",
    "total_amount_last_30d":    "ACCOUNT: Total INR sent in the last 30 days.",
    "amount_zscore_30d":        "ACCOUNT: Z-score of this amount vs 30-day rolling mean. >3 = anomalous spike.",
    "dormancy_flag":            "ACCOUNT: 1 = >30 days since last transaction from this account.",
    "txn_velocity_cumulative":  "ACCOUNT: Cumulative count of all transactions from this account.",

    # ── CUSTOMER-LEVEL VELOCITY (per customer_id, across ALL accounts) ────
    "cust_txn_count_last_1hr":     "CUSTOMER: Transactions across all accounts in last 1 hour.",
    "cust_txn_count_last_24hr":    "CUSTOMER: Transactions across all accounts in last 24 hours.",
    "cust_total_amount_last_24hr": "CUSTOMER: Total INR across all accounts in last 24 hours.",
    "cust_txn_count_last_7d":      "CUSTOMER: Transaction count across all accounts in last 7 days.",
    "cust_total_amount_last_7d":   "CUSTOMER: Total INR across all accounts in last 7 days.",
    "cust_txn_count_last_30d":     "CUSTOMER: Transaction count across all accounts in last 30 days.",
    "cust_total_amount_last_30d":  "CUSTOMER: Total INR across all accounts in last 30 days.",
    "cust_unique_accounts_30d":    "CUSTOMER: Number of unique accounts used (cumulative proxy).",

    # ── RULE ENGINE AGGREGATES ─────────────────────────────────────────────
    "rule_trigger_count":       "Total AML rules triggered by this transaction. Higher = more suspicious.",
    "max_rule_severity":        "Severity (1–3) of the most severe rule triggered. 0 if none.",
    "weighted_rule_score":      "Sum of severities of all triggered rules. Overall suspicion weight.",
}

cat_df = pd.DataFrame([
    {"column_name": k, "description": v,
     "is_pk_fk": "yes" if k in ("transaction_id","customer_id","sender_account_id",
                                 "receiver_account_id","beneficiary_id","device_id") else "no"}
    for k, v in FULL_CATALOGUE.items()
])
cat_df.to_csv(OUTPUT_DIR / "transaction_feature_catalogue.csv", index=False)
print(f"✅ Feature catalogue saved: {len(cat_df)} features documented")
print()
cat_df

✅ Feature catalogue saved: 65 features documented



,column_name,description,is_pk_fk
0,transaction_id,"PK. Unique transaction identifier. Format: T0,...",yes
1,customer_id,FK → customers.customer_id. Denormalised for s...,yes
2,sender_account_id,FK → accounts.account_id. Account that initiat...,yes
3,receiver_account_id,FK → accounts.account_id. NULLABLE. Set for in...,yes
4,beneficiary_id,FK → beneficiaries.beneficiary_id. NULLABLE. S...,yes
...,...,...,...
60,cust_total_amount_last_30d,CUSTOMER: Total INR across all accounts in las...,no
61,cust_unique_accounts_30d,CUSTOMER: Number of unique accounts used (cumu...,no
62,rule_trigger_count,Total AML rules triggered by this transaction....,no
63,max_rule_severity,Severity (1–3) of the most severe rule trigger...,no


In [30]:
cat_df.to_excel(OUTPUT_DIR / "transaction_feature_catalogue.xlsx", index=False)

In [31]:
# ── CELL 16: PK / FK Integrity Validation ────────────────────────────────

print("══════════════════════════════════════════════════════════════")
print("  FK INTEGRITY REPORT")
print("══════════════════════════════════════════════════════════════")

checks = {
    "transaction_id is unique"                     : final_txns["transaction_id"].is_unique,
    "customer_id → customers FK (no orphans)"      : final_txns["customer_id"].isin(customers["customer_id"]).all(),
    "sender_account_id → accounts FK (no orphans)" : final_txns["sender_account_id"].isin(accounts["account_id"]).all(),
    "device_id → devices FK (no orphans)"          : final_txns["device_id"].isin(devices["device_id"]).all(),
    "receiver_account_id XOR beneficiary_id"       : (
        (final_txns["receiver_account_id"].notna() & final_txns["beneficiary_id"].notna()).sum() == 0 and
        (final_txns["receiver_account_id"].isna()  & final_txns["beneficiary_id"].isna()).sum() == 0
    ),
}

all_ok = True
for check, passed in checks.items():
    icon = "✅" if passed else "❌"
    print(f"  {icon}  {check}")
    if not passed:
        all_ok = False

print()
print(f"  Overall: {'✅ ALL CHECKS PASSED' if all_ok else '❌ FAILURES DETECTED'}")

══════════════════════════════════════════════════════════════
  FK INTEGRITY REPORT
══════════════════════════════════════════════════════════════
  ✅  transaction_id is unique
  ✅  customer_id → customers FK (no orphans)
  ✅  sender_account_id → accounts FK (no orphans)
  ✅  device_id → devices FK (no orphans)
  ✅  receiver_account_id XOR beneficiary_id

  Overall: ✅ ALL CHECKS PASSED


In [32]:
# ── CELL 17: Fraud Typology Breakdown ────────────────────────────────────

print("══════════════════════════════════════════════════════════════")
print("  FRAUD TYPOLOGY SUMMARY")
print("══════════════════════════════════════════════════════════════")

fraud_only = final_txns[final_txns["label"] == 1]

print(f"  Total rows      : {len(final_txns):,}")
print(f"  Fraud rows      : {len(fraud_only):,} ({len(fraud_only)/len(final_txns)*100:.2f}%)")
print(f"  Legit rows      : {len(final_txns)-len(fraud_only):,}")
print()

breakdown = final_txns[final_txns["label"]==1]["fraud_type"].value_counts()
for ftype, cnt in breakdown.items():
    pct = cnt / len(fraud_only) * 100
    bar = "█" * int(pct / 2)
    print(f"  {ftype:<20} {cnt:>7,}  ({pct:>5.1f}%)  {bar}")

print()
print("  Velocity by fraud type (mean values):")
vel_summary = final_txns.groupby("fraud_type").agg(
    avg_amount=("amount", "mean"),
    avg_1hr_txns=("txn_count_last_1hr", "mean"),
    avg_24hr_txns=("txn_count_last_24hr", "mean"),
    avg_night=("is_night", "mean"),
    avg_high_risk_bene=("high_risk_beneficiary", "mean"),
).round(2)
print(vel_summary.to_string())

══════════════════════════════════════════════════════════════
  FRAUD TYPOLOGY SUMMARY
══════════════════════════════════════════════════════════════
  Total rows      : 86,992
  Fraud rows      : 11,909 (13.69%)
  Legit rows      : 75,083

  mule_ring              4,800  ( 40.3%)  ████████████████████
  layering               2,700  ( 22.7%)  ███████████
  smurfing               2,387  ( 20.0%)  ██████████
  ATO                    2,000  ( 16.8%)  ████████
  identity_fraud            22  (  0.2%)  

  Velocity by fraud type (mean values):
                avg_amount  avg_1hr_txns  avg_24hr_txns  avg_night  avg_high_risk_bene
fraud_type                                                                            
ATO               50361.62          1.05           2.02       0.50                1.00
identity_fraud    57759.90          1.00           1.73       0.59                1.00
layering           9525.10          1.02           1.58       0.52                0.00
mule_ring         

In [33]:
final_txns.head()

,timestamp,transaction_id,customer_id,sender_account_id,receiver_account_id,beneficiary_id,device_id,amount,channel,debit_credit,...,rule_very_high_risk_offshore,rule_low_kyc_offshore,rule_low_income_large_txn,rule_vpn_offshore,rule_emulator_crypto,rule_new_account_offshore,rule_new_acct_high_cust_velocity,rule_trigger_count,max_rule_severity,weighted_rule_score
0,2025-09-01 02:14:05,T67865,C0,A1353,A750,None,D29,917.51,web,debit,...,0,0,0,0,0,0,0,1,2,2
1,2025-09-01 16:46:27,T55261,C0,A1100,A1165,None,D202,1152.79,web,debit,...,0,0,0,0,0,0,0,0,0,0
2,2025-09-03 11:28:13,T67895,C0,A1353,None,B156,D29,332.39,branch,debit,...,0,0,0,0,0,0,0,2,3,5
3,2025-09-03 11:45:58,T67877,C0,A1353,None,B271,D29,906.33,web,debit,...,0,0,0,0,0,0,0,0,0,0
4,2025-09-04 12:46:13,T55267,C0,A1100,None,B382,D202,147.78,branch,debit,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# ── CELL 18: Save Final Transactions ─────────────────────────────────────

final_txns.to_csv(OUTPUT_DIR / "transactions.csv", index=False)

size_kb = (OUTPUT_DIR / "transactions.csv").stat().st_size / 1024
print(f"✅ transactions.csv saved")
print(f"   {len(final_txns):,} rows × {final_txns.shape[1]} cols | {size_kb:,.0f} KB")
print()
print("Quick join recipes:")
print()
print("  # Transactions + customer profile (single key):")
print("  transactions.merge(customers, on='customer_id')")
print()
print("  # Transactions + account details:")
print("  transactions.merge(accounts, left_on='sender_account_id', right_on='account_id')")
print()
print("  # Transactions + device info:")
print("  transactions.merge(devices, on='device_id')")
print()
print("  # External transfers only + beneficiary info:")
print("  transactions[transactions['beneficiary_id'].notna()].merge(beneficiaries, on='beneficiary_id')")
print()
print("All outputs in ./outputs/:")
for f in sorted(OUTPUT_DIR.glob("*.csv")):
    print(f"  📄 {f.name}")

✅ transactions.csv saved
   86,992 rows × 109 cols | 33,857 KB

Quick join recipes:

  # Transactions + customer profile (single key):
  transactions.merge(customers, on='customer_id')

  # Transactions + account details:
  transactions.merge(accounts, left_on='sender_account_id', right_on='account_id')

  # Transactions + device info:
  transactions.merge(devices, on='device_id')

  # External transfers only + beneficiary info:
  transactions[transactions['beneficiary_id'].notna()].merge(beneficiaries, on='beneficiary_id')

All outputs in ./outputs/:
  📄 accounts.csv
  📄 beneficiaries.csv
  📄 customer_feature_catalogue.csv
  📄 customers.csv
  📄 devices.csv
  📄 transaction_feature_catalogue.csv
  📄 transactions.csv
